# Gallery Example: Hyper/M/1 Reentrant Queue

This example demonstrates a HyperExp/M/1 queueing system with reentrant routing:
- **Arrivals**: Hyperexponential distribution
- **Service**: Exponential distribution
- **Servers**: 1 server
- **Scheduling**: FCFS
- **Reentrant**: Jobs switch class and pass through the queue again

In [ ]:
from line_solver import *
GlobalConstants.set_verbose(VerboseLevel.STD)

In [ ]:
def gallery_hypm1_reentrant():
    """Create Hyper/M/1-Reentrant queueing model"""
    model = Network('Hyper/M/1-Reentrant')
    
    # Block 1: nodes
    source = Source(model, 'Source')
    queue = Queue(model, 'Queue', SchedStrategy.FCFS)
    sink = Sink(model, 'Sink')
    
    # Block 2: classes
    oclass1 = OpenClass(model, 'Class1')
    oclass2 = OpenClass(model, 'Class2')
    
    # HyperExp arrival (SCV=4) for Class1, Class2 is disabled at source
    source.set_arrival(oclass1, HyperExp.fit_mean_and_scv(1, 4))
    source.set_arrival(oclass2, Disabled())
    
    # Exponential service for both classes
    queue.set_service(oclass1, Exp(2))
    queue.set_service(oclass2, Exp(3))
    
    # Block 3: topology with class switching (reentrant)
    P = model.init_routing_matrix()
    P.add_route(oclass1, source, queue, 1.0)
    P.add_class_switch(oclass1, oclass2, queue, queue, 1.0)
    P.add_route(oclass2, queue, sink, 1.0)
    model.link(P)
    
    return model

# Create the model
model = gallery_hypm1_reentrant()

## About Hyperexponential Arrivals

The hyperexponential distribution:
- Is a mixture of exponentials with different rates
- Has SCV ≥ 1 (more variable than exponential)
- Models bursty arrival patterns

With SCV = 4:
- **Mean**: 1.0
- **Variance**: 4.0 (4 times the mean squared)
- **CV (coefficient of variation)**: 2.0

This high variability leads to periods of heavy congestion.

In [ ]:
# Solve with multiple solvers
print("\n=== Solver Results ===")

# CTMC Solver
solver_ctmc = CTMC(model, cutoff=15)
avg_table_ctmc = solver_ctmc.avg_table()
print("\nCTMC Solver:")
print(avg_table_ctmc)

# Fluid Solver
solver_fluid = FLD(model)
avg_table_fluid = solver_fluid.avg_table()
print("\nFluid Solver:")
print(avg_table_fluid)